# Import Libraries

In [1]:
from sklearn.cluster import AgglomerativeClustering, KMeans, HDBSCAN
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder

# Background

Data Source: https://mesonet.agron.iastate.edu/request/download.phtml

Dataset contains a list of all stations with automated airport weather observations that had US domestic commercial travel in 2022. Contains ASOS/AWOS sensor data, also known as METAR data. Originally contained data from all 50 US States plus the US Virgin Islands, Puerto Rico and US Pacific terriroties such as Guam and American Samoa. This has been reduced to only the 50 states as including the smaller locations would make the geospatial plotting difficult.

Along with each station's name, state and identifying codes in three separate formats, also includes latitude/longitude data and elevation of the airfield data.
The latitude/longitude data will be clustered to see how different clustering algorithms will group these stations and compare that to the geographic state boundaries.

# Setup Data

Import data, then remove null values.

Remove all stations outside of CONUS (U.S. Virgin Islands, Puerto Rico, American Samoa)

Set a display name for plotting use later.

Finally, set X as latitude,longitude coordinate pairs. Set y as the state.

In [2]:
df = pd.read_csv('Stations.csv')
df.dropna(inplace=True)
df = df[~df['AIRPORT_STATE_CODE'].isin(['VI', 'PR', 'TT'])].reset_index()
df['AIRPORT'] = df['IATA'].str.cat(df['DISPLAY_AIRPORT_NAME'], sep=' ')
X = df.filter(['LATITUDE', 'LONGITUDE'])
y = df.filter(['AIRPORT_STATE_CODE'])
names = df.filter(['AIRPORT'])

View data

In [3]:
X

,LATITUDE,LONGITUDE
0,56.801388,-132.946100
1,61.174168,-149.998060
2,55.354168,-131.711100
3,64.510560,-165.444720
4,60.778610,-161.837220
...,...,...
363,42.905834,-106.463610
364,44.520280,-109.023890
365,44.348890,-105.539444
366,43.064167,-108.459724


In [4]:
y

,AIRPORT_STATE_CODE
0,AK
1,AK
2,AK
3,AK
4,AK
...,...
363,WY
364,WY
365,WY
366,WY


Label encode states to be able to compare to labels

In [5]:
le = LabelEncoder()
y_true = le.fit_transform(y['AIRPORT_STATE_CODE'].ravel())
y_true

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  1,  1,  1,  1,  1,  2,  2,  2,  2,  3,  3,  3,  3,  3,  3,
        3,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,
        6,  7,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,
        8,  8,  8,  8,  9,  9,  9,  9,  9,  9,  9, 10, 10, 10, 10, 10, 11,
       11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13,
       13, 13, 13, 13, 13, 14, 14, 14, 14, 15, 15, 15, 15, 15, 15, 15, 16,
       16, 16, 16, 16, 17, 17, 17, 17, 17, 17, 17, 18, 18, 18, 18, 18, 19,
       19, 19, 20, 20, 20, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21,
       21, 21, 21, 22, 22, 22, 22, 22, 22, 22, 22, 23, 23, 23, 23, 23, 23,
       23, 23, 24, 24, 24, 24, 24, 25, 25, 25, 25, 25, 25, 25, 25, 26, 26,
       26, 26, 26, 26, 26, 26, 26, 26, 27, 27, 27, 27, 27, 27, 27, 27, 28,
       28, 28, 28, 28, 28

In [6]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 368 entries, 0 to 367
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   LATITUDE   368 non-null    float64
 1   LONGITUDE  368 non-null    float64
dtypes: float64(2)
memory usage: 5.9 KB


368 Values in the dataset.

In [7]:
X.describe()

,LATITUDE,LONGITUDE
count,368.000000,368.000000
mean,39.411596,-97.741625
std,7.669959,20.059627
min,19.720278,-176.642500
25%,34.387360,-109.034237
50%,39.478335,-93.395415
75%,43.160276,-82.551317
max,71.284720,-68.044720


Plotting function to build interactive geospatial scatter plot using plotly

In [8]:
def plot_clusters(model):
    plot_df = X.copy()
    plot_df.insert(0, 'Label', pd.Series(model.labels_))
    plot_df.insert(0, 'Airport', names['AIRPORT'])
    fig = px.scatter_geo(plot_df, lat='LATITUDE', lon='LONGITUDE', symbol='Label', scope='usa', hover_name='Airport', height=700)
    fig.update_traces(marker={'size': 15})
    return fig

## K means

Build a model using k=50 to build 50 clusters using sklearn KMeans

In [9]:
model_km = KMeans(n_clusters=50, n_init='auto').fit(X)
model_km.labels_

array([12,  7, 12,  2, 17,  7, 17, 12,  7,  2, 17, 12, 12, 17, 12, 21, 12,
       24,  7, 13, 39, 30, 13, 30, 31, 26, 26, 31, 38, 38, 38, 27, 38, 38,
       27, 41, 36, 41, 36, 41, 27,  9, 36, 36, 41, 36, 27, 41, 41, 36, 41,
       36, 36, 36,  9, 41, 20, 16, 16, 16, 16, 16, 34, 16, 16, 16, 16, 16,
        5, 40, 30, 30,  8, 47,  8, 47,  8,  8, 30,  8,  8,  8, 47, 47, 30,
        8,  8,  8, 47, 35, 30, 30, 47, 35, 18, 47,  3,  3,  3,  3,  3, 14,
       44, 14, 14, 14, 14, 14, 22, 22, 43,  4,  4,  4, 25,  1, 25, 25, 25,
       25, 25, 25, 25, 25,  1,  1, 23, 25,  6,  6,  6,  6,  6,  6,  6,  1,
       23,  1, 23,  1, 26, 26, 39, 39, 26, 39, 26,  5,  5,  5,  5,  5, 28,
       28, 28,  5,  5,  5, 45, 29, 45, 29, 45, 29, 29, 29, 29, 29, 29, 29,
       29, 45, 29, 19, 19, 19, 19, 14, 14, 19, 19, 31, 31,  1, 31, 31, 31,
       31,  1, 39, 39, 39, 13, 39, 46, 22,  0, 46, 46, 46, 46, 46, 37, 37,
       37, 37, 37, 35, 18, 37, 37, 37, 48, 48, 48, 48, 44, 19, 48, 19,  6,
        6,  6, 34,  6,  6

Calculate accuracy

In [10]:
accuracy_score(y_true, model_km.labels_)

0.03260869565217391

Plot Map

In [11]:
plot_clusters(model_km).show()

## Agglomerative

Build a base model using n_clusters=50 to build 50 clusters using sklearn AgglomerativeClustering

In [12]:
model_agg = AgglomerativeClustering(n_clusters=50).fit(X)
model_agg.labels_

array([46,  2, 46, 34,  9,  2,  9, 16,  2, 34,  2, 16, 46,  9, 16, 45, 16,
       41, 31, 26, 10, 26, 26, 26,  4,  4, 15,  4,  3,  3,  3,  3,  3,  3,
        3,  8,  8,  8, 30,  8, 30,  8,  8, 30,  8,  8, 30,  6,  8, 30,  8,
       30,  8, 30,  8,  8,  6, 49, 36,  7, 49, 49, 36, 49, 49, 49,  7, 36,
       12, 17, 26, 26,  1,  1,  1,  1,  1,  1, 26,  1,  1,  1,  1,  1, 26,
        1,  1,  1,  1, 22, 26, 26, 26, 22, 26,  1, 19, 19, 19, 19, 19, 14,
       42, 14, 14, 14, 14, 14, 13, 13, 43, 20, 20, 20, 25, 38, 25, 25, 25,
       25, 25, 25, 14, 25, 38,  0,  0,  0, 11, 11, 11, 11, 11, 11, 11, 38,
       23, 38, 23, 23, 10, 15, 10, 10, 15, 10, 10, 12, 12, 12, 12, 12, 17,
        5,  5, 47, 47, 12, 29,  0, 29,  0, 29,  0,  0,  0,  0,  0,  0,  0,
        0, 29,  0, 35, 35, 35, 35, 14, 14, 35, 35,  4,  4, 38,  4,  4,  4,
        4, 38, 10, 10, 10, 10, 10, 32, 32, 27, 32, 32, 32, 32, 32, 37, 37,
       22, 22, 37, 22, 44, 22, 22, 37, 21, 21, 21, 40, 40, 40, 21, 40, 11,
       42, 11, 36, 11, 42

Calculate Accuracy

In [13]:
accuracy_score(y_true, model_agg.labels_)

0.02717391304347826

Use some grid search to find best results

In [14]:
linkages = ['ward', 'complete', 'average', 'single']
metrics = ['euclidean', 'l1', 'l2', 'manhattan', 'cosine']
settings = []
results = []
models = []
for linkage in linkages:
    for metric in metrics:
        try:
            model_agg_test = AgglomerativeClustering(n_clusters=50,linkage=linkage,metric=metric).fit(X)
            settings.append([linkage, metric])
            results.append(accuracy_score(y_true, model_agg_test.labels_))
            models.append(model_agg_test)
        except:
            pass
data = {'setting': settings, 'result': results, 'model': models}
metrics_df = pd.DataFrame.from_dict(data)
metrics_df.sort_values(by='result', ascending=False).reset_index(drop=True)

,setting,result,model
0,"[complete, euclidean]",0.032609,"AgglomerativeClustering(linkage='complete', me..."
1,"[complete, l1]",0.032609,"AgglomerativeClustering(linkage='complete', me..."
2,"[complete, l2]",0.032609,"AgglomerativeClustering(linkage='complete', me..."
3,"[complete, manhattan]",0.032609,"AgglomerativeClustering(linkage='complete', me..."
4,"[single, euclidean]",0.032609,"AgglomerativeClustering(linkage='single', metr..."
5,"[single, l2]",0.032609,"AgglomerativeClustering(linkage='single', metr..."
6,"[ward, euclidean]",0.027174,"AgglomerativeClustering(metric='euclidean', n_..."
7,"[complete, cosine]",0.024457,"AgglomerativeClustering(linkage='complete', me..."
8,"[single, cosine]",0.010870,"AgglomerativeClustering(linkage='single', metr..."
9,"[average, euclidean]",0.008152,"AgglomerativeClustering(linkage='average', met..."


Plot Map

In [15]:
plot_clusters(metrics_df.model[0]).show()

## HDBSCAN

Try using sklearn HDBSCAN. Using min_cluster_size=2 gives 100+ clusters, using min_cluster_size=3 reduces this to 3 clusters.

In [16]:
model_hdb = HDBSCAN(min_cluster_size=2).fit(X)
model_hdb.labels_

array([  1,   8,   1,   3,   9,  -1,   9,   2,   8,   3,   8,   2,   1,
         9,   2,  -1,  -1,  -1,  -1,  68,  93,  68,  68,  94,  50,  -1,
        56,  50,  -1,   6,   7,  -1,   7,   6,   4,  20,  26,  20,  27,
        20,  27,  22,  26,  27,  20,  26,  27,  19,  20,  27,  20,  27,
        26,  27,  22,  20,  19,  37,  31,  -1,  36,  37,  31,  37,  37,
        36,  -1,  31, 100, 107,  94,  94,  58,  78,  84,  78,  -1,  83,
        94,  83,  84,  58,  86,  -1,  82,  78,  83,  58,  78,  87,  82,
        82,  82,  88,  82,  86,   0,   0,   0,   0,   0,  70,  47,  70,
        64,  69,  69,  -1,  28,  28,  13,  -1,  23,  23,  63,  57,  71,
        71,  72,  63,  63,  63,  64,  63,  62,  -1,  67,  67,  42,  45,
        42,  42,  45,  45,  42,  53,  66,  62,  66,  66,  -1,  56,  91,
        91,  90,  91,  -1, 100, 100, 101, 101, 101,  -1, 104, 104,  -1,
        -1, 100,  77,  67,  77,  40,  77,  40,  67,  40,  67,  67,  67,
        67,  67,  -1,  40,  -1,  41,  61,  41,  61,  -1,  41,  6

Plot Map

In [17]:
plot_clusters(model_hdb).show()